In [71]:
!pip install ortools


tutaj przypadkowe dystanse, pickup/delivery points i demands, także time matrix (przyjmuję tymczasowo prędkość 100 m/min) i time windows

In [72]:
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


n=50 #num of clients
nv=7 #num of vehicles
cap=3
vc=[cap]*nv #capacities

N=[i for i in range(1,n+1)] #clients
V=[0]+N #verticles with central point
x=np.random.rand(len(V))*1000
y=np.random.rand(len(V))*1000

p=[(i,j) for i in V for j in V ] #all posible routes  
d={(i,j): int(np.hypot([x[i]-x[j]],[y[i]-y[j]])[0]) for (i,j) in p}

distmat=list()

for i in range(n+1):
  distances=list()
  for j in range(n+1):
    distances.append(d[(i,j)])
  distmat.append(distances)

timemat=np.ceil(np.array(distmat,dtype=int)/100)
timemat=np.array(timemat,dtype=int)




indexes=np.arange(1,n+1)
np.random.shuffle(indexes) #randomize indexes
pickups=indexes[0:int((n+1)/2)] #cut indexes in half to create random pickup and delivery points
deliveries=indexes[int((n+1)/2):(n+1)]

pnd=list()
for i in range(len(pickups)):
  pnd.append([pickups[i],deliveries[i]])



demands=np.zeros(len(pickups)*2+1)

for i in range(len(pickups)):
  demands[pickups[i]]+=np.random.randint(1,cap+1)
  demands[deliveries[i]]+=(demands[pickups[i]]*(-1))


timewindows=[(0,5)]*(n+1)
timewindows1=list()
for i in range(len(pickups)):
  t1=np.random.randint(10,61)
  timewindows[pickups[i]]=(t1,t1+5)
  timewindows[deliveries[i]]=(t1+5,t1+25)

for i in timewindows:
  timewindows1.append(list(map(int,i)))


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distmat
    data['time_matrix'] = timemat
    data['time_windows'] = timewindows1
    data['pickups_deliveries'] = pnd
    data['demands'] = demands
    data['vehicle_capacities'] = vc
    data['num_vehicles'] = nv
    data['depot'] = 0
    return data

In [73]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    

    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)

        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def solutionstr(data, manager, routing, solution):
    #Create string with solution
    
    solution_string='Objective: '+str(solution.ObjectiveValue())+'\n'
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n\n'.format(route_load)
        solution_string+=plan_output
        total_distance += route_distance
        total_load += route_load
    solution_string+=('Total distance of all routes: '+str(total_distance)+' m\n')
    solution_string+=('Total load of all routes: '+str(total_load)+'\n')
    return solution_string

Main pickup n delivery z capacity constraints, time limit 100 s

In [75]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <=
            distance_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    search_parameters.time_limit.FromSeconds(100)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print(solutionstr(data, manager, routing, solution))

if __name__ == '__main__':
    main()

Objective: 284387
Route for vehicle 0:
 0 Load(0.0) ->  37 Load(2.0) ->  2 Load(0.0) ->  30 Load(3.0) ->  45 Load(0.0) ->  40 Load(2.0) ->  10 Load(3.0) ->  31 Load(1.0) ->  6 Load(0.0) ->  35 Load(2.0) ->  17 Load(0.0) ->  0 Load(0.0)
Distance of the route: 2384m
Load of the route: 0.0

Route for vehicle 1:
 0 Load(0.0) ->  12 Load(1.0) ->  8 Load(2.0) ->  43 Load(1.0) ->  5 Load(2.0) ->  7 Load(1.0) ->  34 Load(3.0) ->  46 Load(1.0) ->  4 Load(0.0) ->  0 Load(0.0)
Distance of the route: 2092m
Load of the route: 0.0

Route for vehicle 2:
 0 Load(0.0) ->  38 Load(2.0) ->  11 Load(0.0) ->  20 Load(1.0) ->  27 Load(0.0) ->  21 Load(3.0) ->  32 Load(0.0) ->  0 Load(0.0)
Distance of the route: 2356m
Load of the route: 0.0

Route for vehicle 3:
 0 Load(0.0) ->  44 Load(2.0) ->  9 Load(3.0) ->  22 Load(1.0) ->  29 Load(3.0) ->  1 Load(2.0) ->  25 Load(3.0) ->  26 Load(2.0) ->  47 Load(0.0) ->  0 Load(0.0)
Distance of the route: 2614m
Load of the route: 0.0

Route for vehicle 4:
 0 Load(0.0) 

Pickup and delivery with time windows and capacity constraints - nie zawsze daje rozwiązanie, trzeba wtedy zwiększać liczbę samochodów itp.

In [76]:
def solutionstrtime(data, manager, routing, solution):
    #Create string with solution
    
    solution_string='Objective: '+str(solution.ObjectiveValue())+'\n'
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Time of the route: {}min\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n\n'.format(route_load)
        solution_string+=plan_output
        total_distance += route_distance
        total_load += route_load
    solution_string+=('Total time of all routes: '+str(total_distance)+' min\n')
    solution_string+=('Total load of all routes: '+str(total_load)+'\n')
    return solution_string

In [77]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)




    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        100000,  # allow waiting time
        100000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))


    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            time_dimension.CumulVar(pickup_index) <=
            time_dimension.CumulVar(delivery_index))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    search_parameters.time_limit.FromSeconds(100)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print(solutionstrtime(data, manager, routing, solution))

if __name__ == '__main__':
    main()

Objective: 213
Route for vehicle 0:
 0 Load(0.0) ->  9 Load(1.0) ->  38 Load(3.0) ->  11 Load(1.0) ->  1 Load(0.0) ->  21 Load(3.0) ->  32 Load(0.0) ->  20 Load(1.0) ->  27 Load(0.0) ->  0 Load(0.0)
Time of the route: 40min
Load of the route: 0.0

Route for vehicle 1:
 0 Load(0.0) ->  10 Load(1.0) ->  40 Load(3.0) ->  31 Load(1.0) ->  6 Load(0.0) ->  35 Load(2.0) ->  17 Load(0.0) ->  0 Load(0.0)
Time of the route: 26min
Load of the route: 0.0

Route for vehicle 2:
 0 Load(0.0) ->  49 Load(3.0) ->  19 Load(0.0) ->  13 Load(3.0) ->  41 Load(0.0) ->  30 Load(3.0) ->  45 Load(0.0) ->  37 Load(2.0) ->  8 Load(3.0) ->  2 Load(1.0) ->  43 Load(0.0) ->  0 Load(0.0)
Time of the route: 39min
Load of the route: 0.0

Route for vehicle 3:
 0 Load(0.0) ->  14 Load(1.0) ->  39 Load(3.0) ->  36 Load(2.0) ->  15 Load(0.0) ->  50 Load(1.0) ->  44 Load(3.0) ->  3 Load(2.0) ->  22 Load(0.0) ->  29 Load(2.0) ->  47 Load(0.0) ->  0 Load(0.0)
Time of the route: 36min
Load of the route: 0.0

Route for vehicle